In [1]:
import numpy as np
import keras

Using Theano backend.


In [2]:
x= "0123456789"*100000

print len(x)

1000000


In [3]:
print set(x)
print list(set(x))

set(['1', '0', '3', '2', '5', '4', '7', '6', '9', '8'])
['1', '0', '3', '2', '5', '4', '7', '6', '9', '8']


In [4]:
dct = ['*'] +list(set(x))
max_features = len(dct)

rev_dct= [(j,i) for i,j in enumerate(dct)]
rev_dct= dict(rev_dct)

In [5]:
print dct
print rev_dct

['*', '1', '0', '3', '2', '5', '4', '7', '6', '9', '8']
{'*': 0, '1': 1, '0': 2, '3': 3, '2': 4, '5': 5, '4': 6, '7': 7, '6': 8, '9': 9, '8': 10}


In [6]:
x= [rev_dct[ch] for ch in x]

In [7]:
n_timestamps=200
x= x[:len(x) - len(x) % n_timestamps]
x= np.array(x, dtype='int32').reshape((-1,n_timestamps))

In [8]:
print x.shape
print x[0][:20]

(5000, 200)
[ 2  1  4  3  6  5  8  7 10  9  2  1  4  3  6  5  8  7 10  9]


In [9]:
y = np.zeros((x.shape[0], x.shape[1], max_features), dtype='int32')
for i in np.arange(x.shape[0]):
    for j in np.arange(x.shape[1]):
        y[i, j, x[i, j]] = 1

In [10]:
print y.shape

(5000, 200, 11)


In [11]:
x = np.roll(y, 1, axis=1)
x[:, 0, :] = 0
x[:, 0, 0] = 1

In [12]:
print x.shape
print x[1][2]

(5000, 200, 11)
[0 1 0 0 0 0 0 0 0 0 0]


In [13]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM

model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=(200,  max_features)))
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(256, return_sequences=True))
model.add(Dense(max_features))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_1 (LSTM)                    (None, 200, 256)      274432      lstm_input_1[0][0]               
____________________________________________________________________________________________________
lstm_2 (LSTM)                    (None, 200, 256)      525312      lstm_1[0][0]                     
____________________________________________________________________________________________________
lstm_3 (LSTM)                    (None, 200, 256)      525312      lstm_2[0][0]                     
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 200, 11)       2827        lstm_3[0][0]                     
___________________________________________________________________________________________

In [14]:
hist = model.fit(x, y, batch_size=64, nb_epoch=1, verbose=2)

INFO (theano.gof.compilelock): Waiting for existing lock by process '78467' (I am process '81119')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/apple/.theano/compiledir_Darwin-16.4.0-x86_64-i386-64bit-i386-2.7.13-64/lock_dir


Epoch 1/1
391s - loss: 1.2390


In [16]:
import numpy
def mnrnd(probs):
    rnd = numpy.random.random()
    for i in xrange(len(probs)):
        rnd -= probs[i]
        if rnd <= 0:
            return i
    return i

sentences = numpy.zeros((128, n_timestamps, max_features))
sentences[:, 0, 0] = 1

# Start sampling char-sequences. At each iteration i the probability over
# the i-th character of each sequences is computed. 
for i in numpy.arange(10):
    probs = model.predict_proba(sentences, verbose=2)[:,i,:]
    # Go over each sequence and sample the i-th character.
    for j in numpy.arange(len(sentences)):
        sentences[j, i+1, mnrnd(probs[j, :])] = 1
sentences = [sentence[1:].nonzero()[1] for sentence in sentences]

# Convert to readable text.
text = []
for sentence in sentences:
    text.append(''.join([dct[word] for word in sentence]))
print text



['6822436688', '4533*02455', '4112475769', '6134249589', '*223455988', '*713155577', '*023255677', '1223757199', '1033566989', '0225466890', '0044456787', '7131456678', '6952434577', '5093355679', '6354*44343', '72344567*9', '164354*593', '9113486789', '9*32345668', '3544104446', '1231575890', '4102447781', '8734544679', '8914455578', '4944534649', '1626743466', '0134462789', '3243467912', '3343524091', '*523324679', '1923457799', '6313314577', '4125245670', '6823345578', '8865544467', '5123357779', '0643455788', '0413546889', '7393474661', '2726444668', '5*31044546', '6412144487', '1642166347', '2412467009', '2975467533', '8*28335668', '3864355556', '2001446678', '4380534556', '8146454766', '7064246778', '0833456771', '7813355578', '2852247770', '2434916330', '03*2366678', '5601333566', '9235497769', '3462135566', '7342545589', '6113356789', '3042967716', '0434507266', '9531355670', '0144676670', '10*3345368', '0633446699', '6*42334567', '4313672365', '3694335668', '9133566780', '3407